In [1]:
# Install core libraries for fine-tuning
%pip install transformers datasets trl peft accelerate

# Install additional dependencies
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install bitsandbytes sentencepiece protobuf

#%pip uninstall -y jinja2
%pip install --user --ignore-installed jinja2>=3.1.0
import jinja2
print(f"Jinja2 version: {jinja2.__version__}")
#!pip install peft evaluate torch numpy --use-deprecated=legacy-resolver

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.11.1 requires trl!=0.19.0,<=0.23.0,>=0.18.2, but you have trl 0.24.0 which is incompatible.
xformers 0.0.32.post2 requires torch==2.8.0, but you have torch 2.7.1+cu118 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Jinja2 version: 3.1.6


In [2]:
#%pip uninstall -y jinja2
#%pip install jinja2>=3.1.0
import jinja2
print(f"Jinja2 version: {jinja2.__version__}")
import json

file = json.load(open("quac5.json", "r"))
custom = json.load(open("custom.json", "r"))
print(file[1]['paragraphs'])

Jinja2 version: 3.1.6
[{'context': "Malayalam is the language spoken by the Malayalis. Malayalam is derived from old Tamil and Sanskrit in the 6th century. For cultural purposes Malayalam and Sanskrit formed a language known as Manipravalam, where both languages were used in an alternating style. Malayalam is the only among the major Dravidian languages without diglossia. This means, that the Malayalam which is spoken does not differ from the written variant. Malayalam is written using the Malayalam script.  Malayalam literature is ancient in origin. The oldest literature works in Malayalam, distinct from the Tamil tradition, is dated between the 9th century and 11th century. Malayalam literature includes the 14th century Niranam poets (Madhava Panikkar, Sankara Panikkar and Rama Panikkar), whose works mark the dawn of both modern Malayalam language and indigenous Keralite poetry. The Triumvirate of poets (Kavithrayam: Kumaran Asan, Vallathol Narayana Menon and Ulloor S. Parameswara Iy

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Ti


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

print(f"Loaded model: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

Skipping import of cpp extensions due to incompatible torch version 2.7.1+cu118 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Loaded model: Qwen/Qwen2.5-1.5B-Instruct
Model parameters: 1,543,714,304


In [5]:
from datasets import Dataset, concatenate_datasets
def format_prompt(example):
    return f"{json.dumps(example, ensure_ascii=False)}"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})
formatted_data = [format_prompt(item) for item in custom]
custom_set = Dataset.from_dict({"text": formatted_data})

# Transform QuAC to conversational format
def transform_quac_to_chat(example):
    """Convert QuAC to professor oral exam format with error handling"""
    
    # Initialize messages with system prompt
    messages = [
        {
            "role": "system",
            "content": "You are a university professor conducting an oral examination. Ask probing questions based on the context, evaluate student responses, and guide them toward deeper understanding through Socratic questioning."
        }
    ]
    
    # Check if 'paragraphs' key exists and is not empty
    if not example.get('paragraphs') or len(example['paragraphs']) == 0:
        # Return empty messages if no paragraphs
        return {"messages": messages}
    
    # Safely get the first paragraph
    first_paragraph = example['paragraphs'][0]
    
    # Add context if it exists
    context = first_paragraph.get('context', '')
    if context:
        messages[0]['content'] += f"\n\nContext: {context}"
    
    # Get Q&A pairs if they exist
    qas = first_paragraph.get('qas', [])
    
    # Add Q&A pairs as conversation
    for qa in qas:
        # Get question (skip if missing)
        question = qa.get('question', '')
        if not question:
            continue
        
        # Professor asks question
        messages.append({
            "role": "assistant",
            "content": question
        })
        
        # Get answers (skip if missing or empty)
        answers = qa.get('answers', [])
        if answers and len(answers) > 0:
            answer_text = answers[0].get('text', '')
            
            # Skip "CANNOTANSWER" responses
            if answer_text and answer_text != "CANNOTANSWER":
                messages.append({
                    "role": "user",
                    "content": answer_text
                })
    
    return {"messages": messages}

# Transform datasets
quac_transformed = dataset.map(transform_quac_to_chat)

print(f"Transformed {len(quac_transformed)} QuAC examples")

# Convert to Dataset
custom_dataset = custom_set.map(transform_quac_to_chat)

# Combine all datasets
combined_dataset = concatenate_datasets([
    quac_transformed,
    custom_dataset
])

print(f"Total training examples: {len(combined_dataset)}")


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

Transformed 76 QuAC examples


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Total training examples: 78


In [6]:
#%pip uninstall -y jinja2
#pip install jinja2>=3.1.0

def format_chat_template(example):
    """Apply chat template for training"""
    # Apply the model's chat template
    formatted = tokenizer.apply_chat_template(
        example['messages'],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": formatted}

# Format all examples
formatted_dataset = combined_dataset.map(format_chat_template)

# Split into train/validation
train_test_split = formatted_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")

# Preview first example
print("\nExample formatted text:")
print(train_dataset[0]['text'][:500])

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Training samples: 70
Validation samples: 8

Example formatted text:
<|im_start|>system
You are a university professor conducting an oral examination. Ask probing questions based on the context, evaluate student responses, and guide them toward deeper understanding through Socratic questioning.<|im_end|>



In [7]:
# Add LoRA adapters
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig

# Configure 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=16,                      # LoRA rank
    lora_alpha=32,             # LoRA alpha
    target_modules=[           # Which layers to apply LoRA to
        "q_proj",
        "k_proj", 
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")

Trainable parameters: 18,464,768 (2.04%)


In [11]:
from trl import SFTTrainer, SFTConfig

# Configure training
training_args = SFTConfig(
    output_dir="./oral_exam_model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    save_total_limit=3,
    fp16=True,                    # Mixed precision training
    optim="paged_adamw_8bit",     # Memory-efficient optimizer
    dataset_text_field="text",
    packing=False,
    report_to="none"              # Change to "wandb" for tracking
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("Training configuration complete!")

Tokenizing train dataset:   0%|          | 0/70 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/70 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Training configuration complete!


In [12]:
# Train the model
# Start training
print("Starting training...")
trainer_stats = trainer.train()

# Save the final model
trainer.save_model("./oral_exam_model_final")
tokenizer.save_pretrained("./oral_exam_model_final")

print("✓ Training complete!")
print(f"Model saved to: ./oral_exam_model_final")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Starting training...


Step,Training Loss
10,1.485400


✓ Training complete!
Model saved to: ./oral_exam_model_final


In [13]:
# Test the fine-tuned model
from transformers import pipeline

# Load your fine-tuned model
pipe = pipeline(
    "text-generation",
    model="./oral_exam_model_final",
    tokenizer=tokenizer,
    device_map="auto"
)

# Test conversation
test_messages = [
    {"role": "system", "content": "You are a university professor conducting an oral examination."},
    {"role": "assistant", "content": "Let's begin. Can you explain the concept of recursion in programming?"}
]

# Generate response
response = pipe(
    tokenizer.apply_chat_template(test_messages, tokenize=False, add_generation_prompt=True),
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True
)

print("Professor's question:", test_messages[-1]['content'])
print("\nModel's response:", response[0]['generated_text'])

Device set to use cuda:0


Professor's question: Let's begin. Can you explain the concept of recursion in programming?

Model's response: <|im_start|>system
You are a university professor conducting an oral examination.<|im_end|>
<|im_start|>assistant
Let's begin. Can you explain the concept of recursion in programming?<|im_end|>
<|im_start|>assistant
Certainly! Recursion is a fundamental technique in computer science where a function calls itself repeatedly to solve problems by breaking them down into smaller instances of the same problem. This approach allows for elegant and concise solutions, particularly useful for tasks like traversing data structures (like trees or graphs) or solving mathematical equations.

To illustrate, consider the factorial function \( n! \), which calculates the product of all positive integers less than or equal to \( n \). Mathematically, it can be defined recursively as:

\[ n! = n \times (n-1)! \]

with the base case being:

\[ 0! = 1 \]

In this context, if we want to compute \(

In [18]:
merged_model = model.merge_and_unload()

# Save merged model
merged_model.save_pretrained("./oral_exam_model_merged")
tokenizer.save_pretrained("./oral_exam_model_merged")

/home/sahammond/.local/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('./oral_exam_model_merged/tokenizer_config.json',
 './oral_exam_model_merged/special_tokens_map.json',
 './oral_exam_model_merged/chat_template.jinja',
 './oral_exam_model_merged/vocab.json',
 './oral_exam_model_merged/merges.txt',
 './oral_exam_model_merged/added_tokens.json',
 './oral_exam_model_merged/tokenizer.json')

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)

In [ ]:
# In Jupyter Notebook, run this in a cell:
from IPython.display import clear_output
clear_output()

# Then go to: Kernel → Restart & Clear Output
# Save the notebook
# Commit to GitHub